In [3]:
# --- Бібліотеки для даної роботи ---
try:
    import numpy, pandas
    print("Бібліотеки вже встановлені. Пропускаємо інсталяцію.")
except ImportError:
    print("Встановлюємо бібліотеки...")
    %pip install numpy pandas

Бібліотеки вже встановлені. Пропускаємо інсталяцію.


## Задача 3. Аналіз виробництва

**Умова:**

Ви керуєте цехом зі збірки FPV-дронів.
У вас три моделі: "Розвідник" ($x_1$), "Камікадзе" ($x_2$) та важкий "Вантажний" дрон ($x_3$).

Кожен із них потребує різного набору комплектуючих: моторів, польотних контролерів та акумуляторів.

В кінці зміни приходить звіт про загальні витрати деталей, але через збій у системі ви втратили дані про те, скільки саме дронів кожного типу було зібрано.

Це класична задача на відновлення даних, яка вирішується через систему лінійних рівнянь $A\mathbf{x} = \mathbf{b}$.

**Відомі витрати на один дрон (Матриця $A$),**
де стовпці — це типи дронів:
* **"Розвідник":** 4 мотори, 1 контролер, 1 акумулятор.
* **"Камікадзе":** 4 мотори, 1 контролер, 2 акумулятори.
* **"Вантажний":** 6 моторів, 2 контролери, 4 акумулятори.

$$
A = \begin{pmatrix}
4 & 4 & 6 \\
1 & 1 & 2 \\
1 & 2 & 4
\end{pmatrix}
$$

**Всього за зміну було витрачено (Вектор $\mathbf{b}$):**
* 460 моторів
* 130 контролерів
* 240 акумуляторів

$$
\mathbf{b} = \begin{pmatrix} 460 \\ 130 \\ 240 \end{pmatrix}
$$

**Використовуючи бібліотеку numpy, напишіть програму та виконайте наступні кроки:**
1.  Сформуйте матрицю коефіцієнтів $A$ та вектор вільних членів $\mathbf{b}$.
2.  Перед розв'язанням перевірте, чи має система єдиний розв'язок. Для цього обчисліть визначник матриці $A$ за допомогою функції `np.linalg.det(A)`. Якщо він не дорівнює нулю - розв'язок існує.
3.  Розв'яжіть систему за допомогою `np.linalg.solve(A, b)` і знайдіть вектор $x$. Це буде кількість зібраних дронів кожного типу.
4.  Виконайте перевірку правильності розрахунків: перемножте матрицю $A$ на знайдений вектор $x$. Результат має збагатися з вектором $b$.
5.  Виведіть кількість дронів кожного типу та результати перевірки.

In [4]:
import numpy as np
import pandas as pd
from IPython.display import display

consumption_matrix = np.array([
    [4, 4, 6],
    [1, 1, 2],
    [1, 2, 4]
])

total_resources_vector = np.array([460, 130, 240])

determinant = np.linalg.det(consumption_matrix)

if abs(determinant) < 1e-9:
    print("Система не має єдиного розв'язку (визначник = 0)")
else:
    drones_count_vector = np.linalg.solve(consumption_matrix, total_resources_vector)
    
    check_resources = np.dot(consumption_matrix, drones_count_vector)
    
    print("Красивий Вивід:")
    print("Результати розрахунку виробництва:")
    
    df_results = pd.DataFrame({
        "Тип дрона": ["Розвідник", "Камікадзе", "Вантажний"],
        "Зібрана кількість": drones_count_vector
    })
    
    df_check = pd.DataFrame({
        "Ресурс": ["Мотори", "Контролери", "Акумулятори"],
        "Фактично витрачено": total_resources_vector,
        "Розрахункова перевірка": check_resources,
        "Статус": ["ОК" if abs(a-b) < 1e-9 else "Помилка" 
                   for a, b in zip(total_resources_vector, check_resources)]
    })

    display(df_results.style.background_gradient(subset=["Зібрана кількість"], cmap="Blues"))
    
    print("\nЗвіт перевірки витрат:")
    display(df_check.style.map(lambda x: 'color: green' if x == 'ОК' else 'color: red', subset=['Статус']))

    print("Технічний вивід:")
    print("-" * 50)
    print(f"Матриця A (Витрати на одиницю):\n{consumption_matrix}")
    print(f"Вектор b (Загальні витрати):\n{total_resources_vector}")
    print(f"Визначник матриці (Детермінант): {determinant:.4f}")
    print(f"Вектор розв'язку x (Кількість дронів): {drones_count_vector}")
    print(f"Перевірка (A * x): {check_resources}")
    print(f"Збіг підтверджено: {np.allclose(total_resources_vector, check_resources)}")

Красивий Вивід:
Результати розрахунку виробництва:


,Тип дрона,Зібрана кількість
0,Розвідник,20.000000
1,Камікадзе,50.000000
2,Вантажний,30.000000



Звіт перевірки витрат:


,Ресурс,Фактично витрачено,Розрахункова перевірка,Статус
0,Мотори,460,460.000000,ОК
1,Контролери,130,130.000000,ОК
2,Акумулятори,240,240.000000,ОК


Технічний вивід:
--------------------------------------------------
Матриця A (Витрати на одиницю):
[[4 4 6]
 [1 1 2]
 [1 2 4]]
Вектор b (Загальні витрати):
[460 130 240]
Визначник матриці (Детермінант): -2.0000
Вектор розв'язку x (Кількість дронів): [20. 50. 30.]
Перевірка (A * x): [460. 130. 240.]
Збіг підтверджено: True


## Аналітичний розв'язок системи рівнянь

Маємо систему лінійних рівнянь:
$$
\begin{cases}
4x_1 + 4x_2 + 6x_3 = 460 \quad (1) \text{ -- мотори} \\
1x_1 + 1x_2 + 2x_3 = 130 \quad (2) \text{ -- контролери} \\
1x_1 + 2x_2 + 4x_3 = 240 \quad (3) \text{ -- акумулятори}
\end{cases}
$$

**Крок 1. Виразимо $x_3$ (Вантажні дрони)**
Помножимо рівняння (2) на 4, щоб зрівняти коефіцієнти при $x_1$ та $x_2$ з рівнянням (1):
$$4 \cdot (x_1 + x_2 + 2x_3) = 4 \cdot 130$$
$$4x_1 + 4x_2 + 8x_3 = 520 \quad (2^*)$$

Віднімемо рівняння (1) від модифікованого ($2^*$):
$$(4x_1 + 4x_2 + 8x_3) - (4x_1 + 4x_2 + 6x_3) = 520 - 460$$
$$2x_3 = 60$$
$$x_3 = 30$$
**Зібрано:** 30 вантажних дронів.

**Крок 2. Знайдемо $x_2$ (Камікадзе)**
Підставимо $x_3 = 30$ у рівняння (2) та (3):
* З (2): $x_1 + x_2 + 2(30) = 130 \Rightarrow x_1 + x_2 = 70 \quad (A)$
* З (3): $x_1 + 2x_2 + 4(30) = 240 \Rightarrow x_1 + 2x_2 = 120 \quad (B)$

Віднімемо рівняння (A) від (B):
$$(x_1 + 2x_2) - (x_1 + x_2) = 120 - 70$$
$$x_2 = 50$$
**Зібрано:** 50 дронів "Камікадзе".

**Крок 3. Знайдемо $x_1$ (Розвідники)**
Підставимо $x_2 = 50$ у рівняння (A):
$$x_1 + 50 = 70$$
$$x_1 = 20$$
**Зібрано:** 20 дронів "Розвідник".

**Відповідь:**
$\mathbf{x} = (20, 50, 30)$.